In [0]:
import pandas as pd

In [7]:
df = pd.read_csv("/content/news.csv")
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [0]:
df2 = pd.get_dummies(df['label'],drop_first=True)

In [9]:
df = pd.concat([df,df2],axis=1)
df.head()

,Unnamed: 0,title,text,label,REAL
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,1


In [0]:
del df['label']

In [11]:
x = df['title']
x.shape

(6335,)

In [0]:
import nltk 
import re
from nltk.corpus import stopwords

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(x)):
    print(i)
    review = re.sub('[^a-zA-Z]',' ',df['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = " ".join(review)
    corpus.append(review)

In [0]:
corpus

['smell hillari fear',
 'watch exact moment paul ryan commit polit suicid trump ralli video',
 'kerri go pari gestur sympathi',
 'berni support twitter erupt anger dnc tri warn',
 'battl new york primari matter',
 'tehran usa',
 'girl horrifi watch boyfriend left facetim',
 'britain schindler die',
 'fact check trump clinton command chief forum',
 'iran reportedli make new push uranium concess nuclear talk',
 'three clinton iowa glimps fire elud hillari clinton campaign',
 'donald trump shockingli weak deleg game somehow got even wors',
 'strong solar storm tech risk today news oct video',
 'way america prepar world war',
 'trump take cruz lightli',
 'women lead differ',
 'shock michel obama hillari caught glamor date rape promot',
 'hillari clinton huge troubl america notic sick thing hidden pictur liberti writer news',
 'iran bill obama like',
 'chart explain everyth need know partisanship america',
 'slipperi slope trump propos ban muslim',
 'episod sunday wire hail deplor special g

In [0]:
from keras.layers import Embedding,LSTM,Dense
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot

In [16]:
voc_size=5000
one_hot_repr = [one_hot(words,voc_size)for words in corpus]
one_hot_repr

[[1276, 1751, 4281],
 [4422, 1848, 1731, 2430, 4823, 4054, 2093, 918, 3028, 2177, 3337],
 [3040, 4269, 3878, 4726, 3565],
 [4764, 1240, 3214, 1811, 535, 4996, 672, 3287],
 [748, 556, 386, 210, 1636],
 [4604, 4719],
 [1168, 2246, 4422, 68, 2683, 4641],
 [1391, 2029, 4144],
 [4775, 2295, 3028, 860, 719, 849, 4089],
 [3560, 4719, 667, 556, 1191, 4669, 251, 1629, 2862],
 [642, 860, 3263, 611, 1333, 4918, 1751, 860, 3486],
 [2253, 3028, 701, 2389, 649, 1155, 3239, 3795, 2442, 2141],
 [3840, 1240, 2571, 2749, 4178, 182, 4514, 1033, 3337],
 [1033, 3927, 4356, 3365, 3444],
 [3028, 2282, 2806, 2726],
 [2242, 2391, 232],
 [951, 4153, 1020, 1751, 4466, 4566, 4538, 3328, 4584],
 [1751, 860, 1061, 1019, 3927, 1385, 4321, 4690, 4533, 2244, 2553, 3067, 4514],
 [3560, 975, 1020, 93],
 [191, 4809, 3207, 1872, 4923, 494, 3927],
 [153, 3387, 3028, 1930, 4384, 3951],
 [1303, 3182, 635, 986, 4205, 998, 2597, 4127, 94],
 [1751, 860, 667, 3593, 1686, 3682, 2489],
 [556, 2645, 360, 3581, 944, 3657, 3757, 443]

In [18]:
sent_length = 20
embedded_docs = pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1276 1751 4281]
 [   0    0    0 ... 3028 2177 3337]
 [   0    0    0 ... 3878 4726 3565]
 ...
 [   0    0    0 ...  499  413 3863]
 [   0    0    0 ... 4043  727 3235]
 [   0    0    0 ... 4741 3028 1636]]


In [20]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(df['REAL'])

In [22]:
x_final.shape,y_final.shape

((6335, 20), (6335,))

In [0]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x_final,y_final,test_size=0.33,random_state=42)

In [25]:
model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=10,batch_size=64)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4244 samples, validate on 2091 samples
Epoch 1/10
4244/4244 [==============================] - 3s 732us/step - loss: 0.6484 - accuracy: 0.6124 - val_loss: 0.5462 - val_accuracy: 0.7599
Epoch 2/10
4244/4244 [==============================] - 2s 559us/step - loss: 0.4091 - accuracy: 0.8242 - val_loss: 0.4809 - val_accuracy: 0.7724
Epoch 3/10
4244/4244 [==============================] - 2s 552us/step - loss: 0.2868 - accuracy: 0.8831 - val_loss: 0.5103 - val_accuracy: 0.7791
Epoch 4/10
4244/4244 [==============================] - 2s 557us/step - loss: 0.2107 - accuracy: 0.9211 - val_loss: 0.5782 - val_accuracy: 0.7747
Epoch 5/10
4244/4244 [==============================] - 2s 560us/step - loss: 0.1644 - accuracy: 0.9413 - val_loss: 0.6400 - val_accuracy: 0.7700
Epoch 6/10
4244/4244 [==============================] - 2s 563us/step - loss: 0.1352 - accuracy: 0.9541 - val_loss: 0.7137 - val_accuracy: 0.7637
Epoch 7/10
4244/4244 [==============================] - 2s 554us/step - loss

In [28]:
from sklearn.metrics import accuracy_score,confusion_matrix
print(confusion_matrix(ytest,model.predict_classes(xtest)))
accuracy_score(model.predict_classes(xtest),ytest)

[[740 331]
 [210 810]]


0.741272118603539